# GCP Support - USGS M2M API Integration

This notebook demonstrates how to use the GCP Support library to find and export Ground Control Points (GCPs) from USGS M2M API for drone imagery processing.

## Overview

This tutorial covers:
1. **Setting up USGS M2M API authentication** - Configure application token
2. **Parsing H3 cells from manifest files** - Extract H3 cell identifiers from input manifests
3. **Finding GCPs using USGS M2M API** - Search for real GCPs from USGS
4. **Filtering GCPs** - Apply quality filters to GCPs
5. **Exporting GCPs** - Export in formats compatible with MetaShape and ArcGIS Pro

## Prerequisites

- Python 3.8+
- USGS M2M API access (request at https://ers.cr.usgs.gov/profile/access)
- USGS application token (create in your profile's "Applications" section)
- All required packages will be installed in the next cell

## Important Notes

- The USGS `/login` endpoint was deprecated in February 2025
- You must use the `/login-token` endpoint with an application token
- GCPs may be embedded in NAIP or other imagery datasets rather than being standalone
- If GCPs are not found, the code will fall back to mock data for demonstration


In [ ]:
# Install required packages
# Note: If you get import errors after running this, you may need to restart the kernel
!pip install -q h3>=3.7.0 requests>=2.31.0 geopandas>=0.14.0 shapely>=2.0.0 pandas>=2.0.0 numpy>=1.24.0 scipy>=1.10.0

print("✓ Packages installed successfully!")
print("⚠️  If you get import errors, restart the kernel: Kernel -> Restart Kernel")


In [ ]:
# Import necessary libraries
import os
import sys
import getpass
from datetime import datetime

# Add the parent directory to the path to import research_gcp_support
notebook_dir = os.getcwd()
script_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

# Try multiple paths to find research_gcp_support
possible_paths = [
    notebook_dir,  # Current directory
    os.path.dirname(notebook_dir),  # Parent directory
    script_dir,  # Script directory
    os.path.dirname(script_dir),  # Parent of script directory
]

for path in possible_paths:
    if os.path.exists(os.path.join(path, 'research_gcp_support')):
        if path not in sys.path:
            sys.path.insert(0, path)
        break

# Verify scipy is installed (required for spatial distribution analysis)
try:
    import scipy
    print(f"✓ scipy version {scipy.__version__} is installed")
except ImportError:
    print("❌ scipy is not installed!")
    print("Please run the installation cell above (cell with !pip install)")
    print("Or run: !pip install scipy>=1.10.0")
    raise

try:
    from research_gcp_support import GCPFinder
    from research_gcp_support.h3_utils import h3_cells_to_bbox
    from research_gcp_support.manifest_parser import parse_manifest, get_h3_cells_from_manifest
    from research_gcp_support.gcp_filter import calculate_spatial_distribution_score, GCPFilter
    print("✓ Imports successful!")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("\nIf you see this error, make sure:")
    print("1. The research_gcp_support library is installed: pip install -e .")
    print("2. All required packages are installed (run the installation cell above)")
    print("3. Or adjust the sys.path in this cell to point to the library location")
    raise


## Step 1: Setup USGS M2M API Authentication

Enter your USGS application token. This token is required to access the M2M API.

**To get an application token:**
1. Log into https://earthexplorer.usgs.gov/
2. Request M2M API access: https://ers.cr.usgs.gov/profile/access
3. Create an application token in your profile's "Applications" section
4. Copy the token and paste it below


In [ ]:
# Get USGS application token
# Option 1: From environment variable (recommended for security)
usgs_token = os.getenv("USGS_APPLICATION_TOKEN")

# Option 2: Prompt for token if not in environment
if not usgs_token:
    print("Enter your USGS application token:")
    print("(You can also set USGS_APPLICATION_TOKEN environment variable)")
    usgs_token = getpass.getpass("USGS Application Token: ").strip()

if not usgs_token:
    raise ValueError("USGS application token is required. Please provide it above or set USGS_APPLICATION_TOKEN environment variable.")

print("✓ USGS application token configured")
print(f"  Token preview: {usgs_token[:10]}...{usgs_token[-4:] if len(usgs_token) > 14 else '***'}")


## Step 2: Setup Output Directory

Create a directory to store all generated GCP files.


In [ ]:
# Create output directory
output_dir = './gcps_output'
os.makedirs(output_dir, exist_ok=True)

print(f"✓ Output directory created: {output_dir}")


## Step 3: Parse Manifest File or Define Bounding Box

You can either:
1. Parse H3 cells from a manifest file, or
2. Define a bounding box directly

The manifest file contains information about drone imagery, including H3 cell identifiers.


In [ ]:
# Option 1: Parse from manifest file
# Uncomment and modify the path if you have a manifest file
# manifest_path = 'input/input-file.manifest'
# if os.path.exists(manifest_path):
#     h3_cells, prefix = parse_manifest(manifest_path)
#     print(f"✓ Parsed manifest: {len(h3_cells)} H3 cell(s) found")
#     print(f"  H3 cells: {h3_cells}")
#     bbox = h3_cells_to_bbox(h3_cells)
# else:
#     print("⚠️  Manifest file not found, using default bounding box")

# Option 2: Define bounding box directly (example: Eastern United States)
# Modify these coordinates for your area of interest
bbox = (
    35.0,   # min_lat
    -85.0,  # min_lon
    45.0,   # max_lat
    -70.0   # max_lon
)

print(f"✓ Using bounding box:")
print(f"  Min Latitude: {bbox[0]:.6f}")
print(f"  Min Longitude: {bbox[1]:.6f}")
print(f"  Max Latitude: {bbox[2]:.6f}")
print(f"  Max Longitude: {bbox[3]:.6f}")
print(f"\n  Full bbox: {bbox}")


## Step 4: Initialize GCPFinder with USGS Credentials

Initialize the GCPFinder with your USGS application token. This will authenticate with the M2M API.


In [ ]:
# Initialize GCPFinder with USGS M2M API credentials
finder = GCPFinder(
    usgs_application_token=usgs_token,
    min_accuracy=1.0,  # Minimum accuracy in meters
    require_photo_identifiable=True,  # Require photo-identifiable GCPs
    min_gcp_threshold=10,  # Minimum GCPs from USGS before searching NOAA
    min_spread_score=None,  # Optional: minimum spatial spread score
    min_confidence_score=None  # Optional: minimum confidence score
)

print("✓ GCPFinder initialized with USGS M2M API credentials")


## Step 5: Find GCPs from USGS

Search for GCPs in the bounding box using the USGS M2M API. The finder will:
1. Search USGS for GCPs (may search NAIP or other datasets)
2. If not enough GCPs found, search NOAA as fallback
3. Filter GCPs based on quality criteria
4. Calculate spatial distribution metrics


In [ ]:
print("Searching for GCPs using USGS M2M API...")
print("=" * 70)
print(f"Bounding box: {bbox}")
print()

# Find GCPs using the bounding box
# This will authenticate with USGS M2M API and search for GCPs
gcps = finder.find_gcps(bbox=bbox, max_results=100)

print()
print(f"✓ Found {len(gcps)} GCPs after filtering")

# Display spatial distribution metrics if available
if finder.last_spatial_metrics and len(gcps) >= 2:
    metrics = finder.last_spatial_metrics
    print(f"\n  Spatial distribution metrics:")
    print(f"    Spread score: {metrics.get('spread_score', 0):.3f} (0-1, higher is better)")
    print(f"    Confidence score: {metrics.get('confidence_score', 0):.3f} (0-1, higher is better)")
    print(f"    Convex hull ratio: {metrics.get('convex_hull_ratio', 0):.3f}")
    print(f"    Grid coverage: {metrics.get('grid_coverage', 0):.3f}")
    print(f"    Average nearest neighbor: {metrics.get('avg_nearest_neighbor', 0):.3f} meters")
    
    if metrics.get('confidence_score', 0) < 0.5:
        print("\n  ⚠️  Warning: Low confidence score detected!")
        print("  The GCPs may be clustered and may not provide good geometric control.")
elif len(gcps) < 2:
    print("\n  ⚠️  Warning: Not enough GCPs for spatial distribution analysis")
    print("  Need at least 2 GCPs to calculate spatial metrics")

# Display sample GCPs
if gcps:
    print(f"\n  Sample GCPs (first 5):")
    for i, gcp in enumerate(gcps[:5]):
        print(f"    {i+1}. {gcp.get('id', 'Unknown')} at ({gcp['lat']:.6f}, {gcp['lon']:.6f}), accuracy: {gcp.get('accuracy', 'N/A')}m")
else:
    print("\n  ⚠️  No GCPs found. This may mean:")
    print("    1. No GCPs available in this area")
    print("    2. GCPs may be embedded in imagery datasets and require extraction")
    print("    3. Check USGS M2M API documentation for dataset-specific GCP access")


## Step 6: Export GCPs for MetaShape and ArcGIS Pro

Export the found GCPs in formats compatible with both MetaShape and ArcGIS Pro.


In [ ]:
if gcps:
    # Export all formats
    prefix = 'usgs_gcps'
    finder.export_all(gcps, output_dir, prefix)
    
    print(f"✓ Exported all formats to {output_dir}/{prefix}_*")
    print("\nGenerated files:")
    files = [f for f in os.listdir(output_dir) if f.startswith(prefix)]
    for file in sorted(files):
        filepath = os.path.join(output_dir, file)
        size = os.path.getsize(filepath)
        print(f"  - {file} ({size} bytes)")
    
    print("\n" + "=" * 70)
    print("✓ Export completed successfully!")
    print("=" * 70)
    print("\nNext steps:")
    print("  - For MetaShape: Import the .txt or .xml files")
    print("  - For ArcGIS Pro: Import the .csv, .geojson, or .shp files")
else:
    print("⚠️  No GCPs to export. Check the search results above.")


## Step 7: Additional Filtering (Optional)

If you want to apply additional filtering criteria, you can use the GCPFilter class directly.


In [ ]:
# Example: Apply stricter filtering
if gcps:
    print("Applying additional filtering...")
    print("-" * 70)
    
    # Create a filter with stricter criteria
    strict_filter = GCPFilter(
        min_accuracy=0.5,  # Stricter accuracy requirement (0.5m)
        require_photo_identifiable=True,
        min_spread_score=0.5,  # Require good spatial distribution
        min_confidence_score=0.6  # Require high confidence
    )
    
    # Filter the GCPs
    filtered_gcps = strict_filter.filter_gcps(gcps, bbox=bbox)
    
    print(f"  Original GCPs: {len(gcps)}")
    print(f"  Filtered GCPs: {len(filtered_gcps)}")
    
    if len(filtered_gcps) >= 2:
        # Get spatial metrics for filtered set
        spatial_metrics = strict_filter.get_spatial_metrics()
        if spatial_metrics:
            print(f"\n  Filtered set spatial distribution:")
            print(f"    Spread score: {spatial_metrics.get('spread_score', 0):.3f}")
            print(f"    Confidence score: {spatial_metrics.get('confidence_score', 0):.3f}")
    
    # Export filtered results
    if filtered_gcps:
        finder.export_all(filtered_gcps, output_dir, 'usgs_gcps_filtered')
        print(f"\n✓ Exported filtered GCPs to {output_dir}/usgs_gcps_filtered_*")
    else:
        print("\n⚠️  No GCPs passed the strict filter criteria")
else:
    print("⚠️  No GCPs available for additional filtering")


## Summary

This notebook demonstrated:
1. ✅ USGS M2M API authentication with application token
2. ✅ Searching for GCPs in a bounding box
3. ✅ Filtering GCPs based on quality criteria
4. ✅ Calculating spatial distribution metrics
5. ✅ Exporting GCPs in multiple formats (MetaShape, ArcGIS Pro)

## File Formats

- **MetaShape**: `.txt` (CSV format) and `.xml` (marker file format)
- **ArcGIS Pro**: `.csv` (simple CSV), `.geojson` (GeoJSON format), `.shp` (shapefile with supporting files)

## Notes

- GCPs may be embedded in NAIP or other imagery datasets rather than being standalone
- If no GCPs are found, the code may fall back to mock data for demonstration
- Spatial distribution metrics help ensure GCPs are well-distributed for bundle adjustment
- Low confidence scores (< 0.5) indicate clustered GCPs that may not provide good geometric control

## Troubleshooting

If you encounter issues:
1. Verify your USGS application token is valid
2. Check that you have M2M API access enabled
3. Verify the bounding box coordinates are correct
4. Check USGS M2M API documentation: https://m2m.cr.usgs.gov/
